# Calculate probability
### 0. Import data and define constants

In [ ]:
import pandas as pd

INPUT_FILE_PATH = "../output/3_combine_chatgpt_draw.json"
OUTPUT_FILE_PATH = "../output/4_calculate_probability.json"
PROBABILITY_OUTPUT_FILE_PATH = "../output/4_calculate_probability_probability.json"

# Various constants with text defined by Abhinav
# These constants are here so that, if Abhinav changes his mind on what the strings will look like, 
# the information is centralized
ALL_ANSWERS = "has all the answers"
ALL_ANSWERS_ROUNDED = "has all the answers when rounded"
SOME_SOLUTION = "has one or more of the answers, but not all of them"
SOME_SOLUTION_ROUNDED = "has one or more of the answers when rounded, but not all of them"
NO_SOLUTION = "says no solution"
INVALID = "invalid"

### 1. Extract data from files into pandas dataframe

In [ ]:
data = pd.read_json(INPUT_FILE_PATH)

In [ ]:
# All null values in Abhinav's dataset will be replaced with the value "invalid" in the previous script
# We count the number of rows that do not have "invalid" as results
def row_is_valid(row):
    return row["valid"]

def number_of_valid_results (data):
    size = data.apply(lambda row : 0 if row_is_valid(row) else 1, axis = 1).sum()
    return size

In [ ]:
def get_base_probability(data):
    return data.apply(lambda row : row["is_correct"], axis=1).sum() / number_of_valid_results(data)

# Calculates the conditional probability for a particular column.
# Furthermore, it also generates several column which is where it is the no. of mathematical features greater than or equal to 1 ... largest
def generate_conditional_probability(data, column, largest):
    print("Conditional for: ", column, "-------------------")
    base_probability = get_base_probability(data)
    
    print("Base probability of success: ", base_probability)

    for i in range(1, largest + 1):
        NEW_COLUMN_NAME = column + "_geq_" + str(i)
        data[NEW_COLUMN_NAME] = data.apply(lambda row : 1 if row[column] >= i else 0, axis=1)
        conditional_size = data.head(number_of_valid_results(data)).apply(lambda row : row[NEW_COLUMN_NAME], axis=1).sum()
        
        if conditional_size > 0:
            conditional_probability = data.apply(lambda row : (row["is_correct"]) * row[NEW_COLUMN_NAME], axis=1).sum() / conditional_size
            print("Probability of success when count is greater than or equal to", str(i), " is: ", conditional_probability)
            print("Support:", conditional_size, "")
        else:
            print("There aren't any of count greater than", i)

    print("\n")

In [ ]:
# Calculate probability of succeeding considering amount of a certain feature
for column in data.columns:
    if column == "sQuestion" or column == "lSolutions" or column == "Template" or column == "lEquations" or column == "iIndex" or column == "Alignment" or column == "Equiv" or column == "is_correct" or column == "valid" or column == "result" or column == "final_answer":
        continue
    print(column)
    generate_conditional_probability(data, column, 8)

In [ ]:
data

In [ ]:
data.to_json(OUTPUT_FILE_PATH)